In [1]:
#importing libraries for RYU package
!pip install ryu

!pip uninstall gunicorn
!pip uninstall eventlet
!pip install gunicorn==20.1.0
!pip install eventlet==0.30.2

#gunicorn --worker-class eventlet -w 1 app:app

from ryu import base
from ryu.base import app_manager
from ryu.controller import ofp_event
from ryu.controller.handler import CONFIG_DISPATCHER, MAIN_DISPATCHER
from ryu.controller.handler import set_ev_cls
from ryu.ofproto import ofproto_v1_3
from ryu.lib.packet import packet
from ryu.lib.packet import ethernet
from ryu.lib.packet import ether_types


class SimpleSwitch13(app_manager.RyuApp):
    OFP_VERSIONS = [ofproto_v1_3.OFP_VERSION]

    def __init__(self, *args, **kwargs):
        super(SimpleSwitch13, self).__init__(*args, **kwargs)
        self.mac_to_port = {}

    @set_ev_cls(ofp_event.EventOFPSwitchFeatures, CONFIG_DISPATCHER)
    def switch_features_handler(self, ev):
        datapath = ev.msg.datapath
        ofproto = datapath.ofproto
        parser = datapath.ofproto_parser
        match = parser.OFPMatch()
        actions = [parser.OFPActionOutput(ofproto.OFPP_CONTROLLER,
                                          ofproto.OFPCML_NO_BUFFER)]
        self.add_flow(datapath, 0, match, actions)

    def add_flow(self, datapath, priority, match, actions, buffer_id=None):
        ofproto = datapath.ofproto
        parser = datapath.ofproto_parser

        inst = [parser.OFPInstructionActions(ofproto.OFPIT_APPLY_ACTIONS,
                                             actions)]
        if buffer_id:
            mod = parser.OFPFlowMod(datapath=datapath, buffer_id=buffer_id,
                                    priority=priority, match=match,
                                    instructions=inst)
        else:
            mod = parser.OFPFlowMod(datapath=datapath, priority=priority,
                                    match=match, instructions=inst)
        datapath.send_msg(mod)

    @set_ev_cls(ofp_event.EventOFPPacketIn, MAIN_DISPATCHER)
    def _packet_in_handler(self, ev):
       
        if ev.msg.msg_len < ev.msg.total_len:
            self.logger.debug("packet truncated: only %s of %s bytes",
                              ev.msg.msg_len, ev.msg.total_len)
        msg = ev.msg
        datapath = msg.datapath
        ofproto = datapath.ofproto
        parser = datapath.ofproto_parser
        in_port = msg.match['in_port']

        pkt = packet.Packet(msg.data)
        eth = pkt.get_protocols(ethernet.ethernet)[0]

        if eth.ethertype == ether_types.ETH_TYPE_LLDP:
            
            return
        dst = eth.dst
        src = eth.src

        dpid = datapath.id
        self.mac_to_port.setdefault(dpid, {})

        self.logger.info("packet in %s %s %s %s", dpid, src, dst, in_port)

        self.mac_to_port[dpid][src] = in_port

        if dst in self.mac_to_port[dpid]:
            out_port = self.mac_to_port[dpid][dst]
        else:
            out_port = ofproto.OFPP_FLOOD

        actions = [parser.OFPActionOutput(out_port)]
        if out_port != ofproto.OFPP_FLOOD:
            match = parser.OFPMatch(in_port=in_port, eth_dst=dst, eth_src=src)
           
            if msg.buffer_id != ofproto.OFP_NO_BUFFER:
                self.add_flow(datapath, 1, match, actions, msg.buffer_id)
                return
            else:
                self.add_flow(datapath, 1, match, actions)
        data = None
        if msg.buffer_id == ofproto.OFP_NO_BUFFER:
            data = msg.data

        out = parser.OFPPacketOut(datapath=datapath, buffer_id=msg.buffer_id,
                                  in_port=in_port, actions=actions, data=data)
        datapath.send_msg(out)


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Found existing installation: gunicorn 20.1.0
Uninstalling gunicorn-20.1.0:
  Would remove:
    /usr/local/bin/gunicorn
    /usr/local/lib/python3.7/dist-packages/gunicorn-20.1.0.dist-info/*
    /usr/local/lib/python3.7/dist-packages/gunicorn/*
Proceed (y/n)? y
  Successfully uninstalled gunicorn-20.1.0
Found existing installation: eventlet 0.30.2
Uninstalling eventlet-0.30.2:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/eventlet-0.30.2.dist-info/*
    /usr/local/lib/python3.7/dist-packages/eventlet/*
Proceed (y/n)? y
  Successfully uninstalled eventlet-0.30.2
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached gunicorn-20.1.0-py3-none-any.whl (79 kB)
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached eventlet-0.30.2-py2.py3-none-any.whl (224 kB)


OSError: ignored

In [2]:
sudo apt-get -o Dpkg::Options::="--force-confmiss" install --reinstall netbase

from ryu import base
from ryu.base import app_manager
from ryu.controller import ofp_event
from ryu.controller.handler import CONFIG_DISPATCHER, MAIN_DISPATCHER
from ryu.controller.handler import set_ev_cls
from ryu.ofproto import ofproto_v1_3
from ryu.lib.packet import packet
from ryu.lib.packet import ethernet
from ryu.lib.packet import ether_types


class SimpleSwitch13(app_manager.RyuApp):
    OFP_VERSIONS = [ofproto_v1_3.OFP_VERSION]

    def __init__(self, *args, **kwargs):
        super(SimpleSwitch13, self).__init__(*args, **kwargs)
        self.mac_to_port = {}

    @set_ev_cls(ofp_event.EventOFPSwitchFeatures, CONFIG_DISPATCHER)
    def switch_features_handler(self, ev):
        datapath = ev.msg.datapath
        ofproto = datapath.ofproto
        parser = datapath.ofproto_parser
        match = parser.OFPMatch()
        actions = [parser.OFPActionOutput(ofproto.OFPP_CONTROLLER,
                                          ofproto.OFPCML_NO_BUFFER)]
        self.add_flow(datapath, 0, match, actions)

    def add_flow(self, datapath, priority, match, actions, buffer_id=None):
        ofproto = datapath.ofproto
        parser = datapath.ofproto_parser

        inst = [parser.OFPInstructionActions(ofproto.OFPIT_APPLY_ACTIONS,
                                             actions)]
        if buffer_id:
            mod = parser.OFPFlowMod(datapath=datapath, buffer_id=buffer_id,
                                    priority=priority, match=match,
                                    instructions=inst)
        else:
            mod = parser.OFPFlowMod(datapath=datapath, priority=priority,
                                    match=match, instructions=inst)
        datapath.send_msg(mod)

    @set_ev_cls(ofp_event.EventOFPPacketIn, MAIN_DISPATCHER)
    def _packet_in_handler(self, ev):
       
        if ev.msg.msg_len < ev.msg.total_len:
            self.logger.debug("packet truncated: only %s of %s bytes",
                              ev.msg.msg_len, ev.msg.total_len)
        msg = ev.msg
        datapath = msg.datapath
        ofproto = datapath.ofproto
        parser = datapath.ofproto_parser
        in_port = msg.match['in_port']

        pkt = packet.Packet(msg.data)
        eth = pkt.get_protocols(ethernet.ethernet)[0]

        if eth.ethertype == ether_types.ETH_TYPE_LLDP:
            
            return
        dst = eth.dst
        src = eth.src

        dpid = datapath.id
        self.mac_to_port.setdefault(dpid, {})

        self.logger.info("packet in %s %s %s %s", dpid, src, dst, in_port)

        self.mac_to_port[dpid][src] = in_port

        if dst in self.mac_to_port[dpid]:
            out_port = self.mac_to_port[dpid][dst]
        else:
            out_port = ofproto.OFPP_FLOOD

        actions = [parser.OFPActionOutput(out_port)]
        if out_port != ofproto.OFPP_FLOOD:
            match = parser.OFPMatch(in_port=in_port, eth_dst=dst, eth_src=src)
           
            if msg.buffer_id != ofproto.OFP_NO_BUFFER:
                self.add_flow(datapath, 1, match, actions, msg.buffer_id)
                return
            else:
                self.add_flow(datapath, 1, match, actions)
        data = None
        if msg.buffer_id == ofproto.OFP_NO_BUFFER:
            data = msg.data

        out = parser.OFPPacketOut(datapath=datapath, buffer_id=msg.buffer_id,
                                  in_port=in_port, actions=actions, data=data)
        datapath.send_msg(out)


SyntaxError: ignored

In [3]:
#all the libraries are imported 
!pip install mininet
from mininet.util import irange,dumpNodeConnections
from mininet.net import Mininet
from mininet.node import Controller, RemoteController, OVSController
from mininet.cli import CLI
from mininet.log import setLogLevel, info
from mininet.link import TCLink

#we created a function 
def TopoCreate():

    net = Mininet( controller= RemoteController )

    info( '*** Adding controller\n' )
    Rc = net.addController(name = 'c0', controller = RemoteController , ip='127.0.0.1' , port=6653 )
#adding Hosts
    h1 = net.addHost( 'h1' )
    h2 = net.addHost( 'h2' )
    h3 = net.addHost( 'h3' )
    h4 = net.addHost( 'h4' )
    h5 = net.addHost( 'h5' )
    h6 = net.addHost( 'h6' )OVSControl

#adding Switches
    s1 = net.addSwitch( 's1' )
    s2 = net.addSwitch( 's2' )
    s3 = net.addSwitch( 's3' )
    s4 = net.addSwitch( 's4' )
    s5 = net.addSwitch( 's5' )
    s6 = net.addSwitch( 's6' )

#adding Links
    net.addLink( h1, s1 )
    net.addLink( h2, s2 )
    net.addLink( h3, s3 )
    net.addLink( h4, s4 )
    net.addLink( h5, s5 )
    net.addLink( h6, s6 )
    net.addLink( s1, s2, cls = TCLink, bw = 10 )
    net.addLink( s1, s3, cls = TCLink, bw = 10 )
    net.addLink( s1, s5, cls = TCLink, bw = 5 )
    net.addLink( s3, s4, cls = TCLink, bw = 5 )
    net.addLink( s5, s6, cls = TCLink, bw = 5 )



    net.start()

    dumpNodeConnections(net.hosts)
    net.pingAll()
   #calculating bandwidth using IPERF given in question 
    a,_=net.iperf((h1,h2))
    b,_=net.iperf((h1,h3))
    c,_=net.iperf((h1,h4))
    d,_=net.iperf((h1,h5))
    e,_=net.iperf((h1,h6))
    f,_=net.iperf((h2,h3))
    g,_=net.iperf((h2,h4))
    h,_=net.iperf((h2,h5))
    i,_=net.iperf((h2,h6))
    j,_=net.iperf((h3,h4))
    k,_=net.iperf((h3,h5))
    l,_=net.iperf((h3,h6))
    m,_=net.iperf((h4,h5))
    n,_=net.iperf((h4,h6))
    o,_=net.iperf((h5,h6))
    #printing costs
    print('*********COSTS*********')
    print("h1		h2       	"+a)
    print("h1           h3              "+b)
    print("h1           h4              "+c)
    print("h1           h5              "+d)
    print("h1           h6              "+e)
    print("h2           h3              "+f)
    print("h2           h4              "+g)
    print("h2           h5              "+h)
    print("h2           h6              "+i)
    print("h3           h4              "+j)
    print("h3           h5              "+k)
    print("h3           h6              "+l)
    print("h4           h5              "+m)
    print("h4           h6              "+n)
    print("h5           h6              "+o)

    CLI(net)
   

if __name__ == '__main__':
    setLogLevel( 'info' )
    TopoCreate()

SyntaxError: ignored